In [2]:
%pip install --upgrade --quiet  langchain langchain-community  langchain-experimental neo4j wikipedia tiktoken yfiles_jupyter_graphs

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.7/301.7 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 45.6 MB/s eta 0:00:00


In [3]:
from langchain_core.runnables import RunnableBranch, RunnableLambda, RunnableParallel, RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import ConfigurableField
from langchain_community.vectorstores import Neo4jVector
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase
from langchain_community.graphs.neo4j_graph import Neo4jGraph
from dotenv import load_dotenv
import os
load_dotenv()


False

In [4]:
NEO4J_URI="neo4j+s://3e7f7f90.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="kkj99K1SbXiAsUYSTMq8Uub_sP66cJ-QdxwHGPTO4U0"

In [5]:
os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD

In [6]:
graph = Neo4jGraph()

<ipython-input-6-6e7385464d00>:1: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph()


In [26]:
from langchain.document_loaders import WikipediaLoader
raw_docs = WikipediaLoader(query = "Mona Lisa").load()

/usr/local/lib/python3.11/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.11/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [27]:
raw_docs

[Document(metadata={'title': 'Mona Lisa', 'summary': 'The Mona Lisa ( MOH-nə LEE-sə; Italian: la Gioconda [la dʒoˈkonda] or Monna Lisa [ˈmɔnna ˈliːza]; French: la Joconde [la ʒɔkɔ̃d]) is a half-length portrait painting by Italian artist Leonardo da Vinci. Considered an archetypal masterpiece of the Italian Renaissance, it has been described as "the best known, the most visited, the most written about, the most sung about, [and] the most parodied work of art in the world." The painting\'s novel qualities include the subject\'s enigmatic expression, monumentality of the composition, the subtle modelling of forms, and the atmospheric illusionism.\nThe painting has been traditionally considered to depict the Italian noblewoman Lisa del Giocondo. It is painted in oil on a white poplar panel. Leonardo never gave the painting to the Giocondo family. It was believed to have been painted between 1503 and 1506; however, Leonardo may have continued working on it as late as 1517. King Francis I of

In [28]:
len(raw_docs)

24

In [29]:
from langchain.text_splitter import TokenTextSplitter
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents = text_splitter.split_documents(raw_docs[:3])

In [30]:
documents[:3]

[Document(metadata={'title': 'Mona Lisa', 'summary': 'The Mona Lisa ( MOH-nə LEE-sə; Italian: la Gioconda [la dʒoˈkonda] or Monna Lisa [ˈmɔnna ˈliːza]; French: la Joconde [la ʒɔkɔ̃d]) is a half-length portrait painting by Italian artist Leonardo da Vinci. Considered an archetypal masterpiece of the Italian Renaissance, it has been described as "the best known, the most visited, the most written about, the most sung about, [and] the most parodied work of art in the world." The painting\'s novel qualities include the subject\'s enigmatic expression, monumentality of the composition, the subtle modelling of forms, and the atmospheric illusionism.\nThe painting has been traditionally considered to depict the Italian noblewoman Lisa del Giocondo. It is painted in oil on a white poplar panel. Leonardo never gave the painting to the Giocondo family. It was believed to have been painted between 1503 and 1506; however, Leonardo may have continued working on it as late as 1517. King Francis I of

In [ ]:
! pip install langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 3.1 MB/s eta 0:00:00


In [42]:
! pip install langchain_neo4j

In [9]:
! pip install langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 6.0 MB/s eta 0:00:00


In [56]:
from langchain_groq import ChatGroq


llm = ChatGroq (
    temperature=0,
    groq_api_key = "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",
    model_name="llama-3.1-70b-versatile"
)

In [32]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer = LLMGraphTransformer(llm=llm)

In [33]:
graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [34]:
graph_documents

[GraphDocument(nodes=[Node(id='Mona Lisa', type='Person', properties={}), Node(id='Leonardo Da Vinci', type='Person', properties={}), Node(id='Lisa Del Giocondo', type='Person', properties={}), Node(id='King Francis I Of France', type='Person', properties={}), Node(id='Vincenzo Peruggia', type='Person', properties={})], relationships=[Relationship(source=Node(id='Mona Lisa', type='Person', properties={}), target=Node(id='Leonardo Da Vinci', type='Person', properties={}), type='CREATED', properties={}), Relationship(source=Node(id='Mona Lisa', type='Person', properties={}), target=Node(id='Lisa Del Giocondo', type='Person', properties={}), type='DEPICTS', properties={}), Relationship(source=Node(id='Mona Lisa', type='Person', properties={}), target=Node(id='King Francis I Of France', type='Person', properties={}), type='OWNED', properties={}), Relationship(source=Node(id='Mona Lisa', type='Person', properties={}), target=Node(id='Vincenzo Peruggia', type='Person', properties={}), type='

In [35]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [36]:
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"

In [37]:
def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    display(widget)
    return widget

In [39]:
from google.colab import output
output.enable_custom_widget_manager()

In [40]:
showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

GraphWidget(layout=Layout(height='800px', width='100%'))

In [43]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_neo4j import Neo4jVector

# Load the Hugging Face model (replace with your desired model)
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

<ipython-input-43-8e3569fcbae4>:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [44]:
from google.colab import output
output.enable_custom_widget_manager()

Support for third party widgets will remain active for the duration of the session. To disable support:

In [45]:
from google.colab import output
output.disable_custom_widget_manager()

In [46]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)

In [48]:
from typing import Tuple, List, Optional

In [49]:
from langchain_core.pydantic_v1 import BaseModel, Field
# Extract entities from text
class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )

In [59]:
llm = ChatGroq (
    temperature=0,
    groq_api_key = "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",
    model_name="llama-3.1-70b-versatile"
)

In [60]:
entity_chain = prompt | llm.with_structured_output(Entities)

In [63]:
entity_chain.invoke({"question": "Who is fuck off freind?"}).names

['fuck off freind']